In [58]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import random
import xgboost as xgb
from sklearn import preprocessing, model_selection
from sklearn.metrics import log_loss
import string
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import  CountVectorizer
from scipy.stats import boxcox
from scipy import stats
from scipy import sparse

In [59]:
processed_train = filter_data_to_columns(pd.read_json('data/train.json'))
y_train = pd.read_json('data/train_interest.json')

In [89]:
def fpreproc_leaky_dataframe(dtrain, dtest):
    train, test = dtrain.data, dtest.data
    ntrain = train.shape[0]
    train_test = pd.concat((train, test), axis=0).reset_index(drop=True)
    y_train = dtrain.labels
    
    # average listing worth
    average_listing_worth = 0 #sum(average_interest) / len(average_interest)
    average_low_frac = sum(y_train.values == 2)[0] * 1.0 / len(y_train)
    average_med_frac = sum(y_train.values == 1)[0] * 1.0 / len(y_train)
    average_high_frac = sum(y_train.values == 0)[0] * 1.0 / len(y_train)
    average_for_counting_strategy = [average_listing_worth, average_low_frac, average_med_frac, average_high_frac]
        
    # add manager worths, building worths, street worths
    divisions = ['manager_id', 'building_id', 'display_address', 'BoroCode', 'NTACode']
    new_col_roots = ['manager', 'building', 'address', 'boro', 'nta']
    for division, new_col_root in zip(divisions, new_col_roots):
        division_ids = np.unique(train[division])
        division_worths = {}
        for division_id in division_ids:            
            interests = y_train[train[division] == division_id].apply(lambda x: 2 - x)
            if len(interests) > 0:
                worth = sum(interests.values)[0] * 1.0 / len(interests)
                low_frac = sum(interests.values == 0)[0] * 1.0 / len(interests)
                med_frac = sum(interests.values == 1)[0] * 1.0 / len(interests)
                high_frac = sum(interests.values == 2)[0] * 1.0 / len(interests)
                division_worths[division_id] = [worth, low_frac, med_frac, high_frac]
        for i, counting_strategy in enumerate(['worths', 'low_frac', 'med_frac', 'high_frac']):
            train_test['{0}_{1}'.format(new_col_root, counting_strategy)] = train_test[division].apply(lambda x: division_worths[x][i] if x in division_worths else average_for_counting_strategy[i])
        
    
    # add price by area
    lat_long_price = train_test[['latitude', 'longitude', 'price_per_bedroom']]
    remove_outliers = (np.abs(stats.zscore(lat_long_price)) < 0.15).all(axis=1)
    lat_long_price = lat_long_price[remove_outliers]
    lat_max, lat_min = max(lat_long_price.latitude), min(lat_long_price.latitude)
    long_max, long_min = max(lat_long_price.longitude), min(lat_long_price.longitude)
    lat_scale, long_scale = lat_max - lat_min, long_max - long_min
    costs = np.zeros((100,100))
    num_listings = np.zeros((100,100))
    for lat, long, price_per_bedroom in lat_long_price.values:
        scaled_lat, scaled_long = int((lat - lat_min) * 99 / lat_scale), int((long - long_min) * 99 / long_scale)
        costs[scaled_lat][scaled_long] += price_per_bedroom
        num_listings[scaled_lat][scaled_long] += 1

    price_by_area = []
    for lat, long, price_per_bedroom in train_test[['latitude', 'longitude', 'price_per_bedroom']].values:
        scaled_lat, scaled_long = int((lat - lat_min) * 99 / lat_scale), int((long - long_min) * 99 / long_scale)
        if scaled_lat < 0 or scaled_lat >= 100 or scaled_long < 0 or scaled_long >= 100:
            price_by_area.append(1)
        elif num_listings[scaled_lat][scaled_long] > 8:
            price_by_area.append(price_per_bedroom / (costs[scaled_lat][scaled_long] / num_listings[scaled_lat][scaled_long]))
        else:
            cost = 0
            num = 0
            for i in range(scaled_lat - 1, scaled_lat + 2):
                for j in range(scaled_long - 1, scaled_long + 2):
                    if i > 0 and i < 100 and j >= 0 and j < 100:
                        cost += costs[i][j]
                        num += num_listings[i][j]
            if num > 8:
                price_by_area.append(price_per_bedroom / (cost / num))
            else:
                price_by_area.append(1)

    train_test['price_by_area'] = price_by_area
    
    
    # try adding real - predicted price
    # Try to predict price for a listing and add real_price - expected_price as a feature
    features_to_use = ['BoroCode', 'NTACode', 'bathrooms', 'bedrooms', 'other_address', 'building_worths', 'manager_worths']
    feature_to_predict = 'price'
    params = {
        'objective': 'reg:linear',
        'booster':'gblinear',
        'lambda': 0,
        'lambda_bias' : 0,
        'alpha': 0.2
    }
    prices = train_test[feature_to_predict]
    remove_outliers = np.abs(prices-prices.mean())<=(3*prices.std())
    dtrain = xgb.DMatrix(data=train_test[remove_outliers][features_to_use], label=train_test[remove_outliers][feature_to_predict])

    bst = xgb.cv(params, dtrain, 10000, 4, early_stopping_rounds=50, verbose_eval=200)
    best_rounds = np.argmin(bst['test-rmse-mean'])
    print (bst['test-rmse-mean'][best_rounds])
    bst = xgb.train(params, dtrain, best_rounds)
    dtrain = xgb.DMatrix(data=train_test[features_to_use])
    expected_price = bst.predict(dtrain)
    train_test['real_minus_expected_price'] = train_test[feature_to_predict] - expected_price
    train_test['real_over_expected_price'] = train_test[feature_to_predict] / expected_price
    
    
    # enumerated streets / price / price per street (/ num bedrooms) / price / price per street (/ num bedrooms + 0.1*bath)
    reasonable_prices = train_test[np.abs(prices-prices.mean())<=(3*prices.std())]
    for split in ['manager_id', 'building_id', 'display_address', 'BoroCode', 'NTACode']:
        unique_labels = np.unique(reasonable_prices[split])
        label_to_price = {}
        label_to_interest = {}
        for label in unique_labels:
            listings = reasonable_prices[reasonable_prices[split] == label]
            if len(listings) > 10:
                label_to_price[label] = sum(listings.price_per_bedroom) / len(listings)
                
            interests = y_train[train[split] == label].apply(lambda x: 2 - x)
            if len(interests) > 5:
                label_to_interest[label] = sum(interests.values)[0] / len(interests)
                
        train_test['price_by_{}'.format(split)] = train_test.apply(lambda x: (x.price_per_bedroom / label_to_price[x[split]]) if x[split] in label_to_price else 1, axis=1)
        train_test['worth_by_{}'.format(split)] = train_test[split].apply(lambda x: label_to_interest[x] if x in label_to_interest else average_listing_worth)
    
    # remove extra
    train_test.drop('price', axis=1, inplace=True)
    
    return train_test[:ntrain], train_test[ntrain:], y_train

def fpreproc_leaky(dtrain, dtest, param):
    train, test, y_train = fpreproc_leaky_dataframe(dtrain, dtest)
    dtrain = xgb.DMatrix(data=train, label=y_train)
    dtest = xgb.DMatrix(data=test)
    return dtrain, dtest, param

In [90]:
current_filter = ['bc_price', 'price_per_bedroom', 'price_per_bathroom', 'price_per_bed_and_bath', 'BoroCode', 'NTACode', 'price', 'latitude', 'price_by_area', 'longitude', 'manager_worths', 'address1', 'real_over_expected_price', 'desc_words_length', 'real_minus_expected_price', 'manager_id', 'Yday', 'building_id', 'building_worths', 'desc_letters_count', 'Day', 'desc_words_count', 'hour', 'bedrooms', 'photos_count', 'features_count', 'Wday', 'sparse_nofee', 'bathrooms', 'sparse_furnished', 'sparse_laundryinbuilding', 'sparse_hardwoodfloors', 'sparse_laundryinunit', 'Month', 'sparse_catsallowed', 'sparse_exclusive', 'street', 'sparse_elevator', 'sparse_prewar', 'sparse_dogsallowed', 'sparse_reducedfee', 'avenue', 'east', 'sparse_doorman', 'sparse_dishwasher', 'other_address', 'sparse_fitnesscenter', 'sparse_privateoutdoorspace', 'sparse_commonoutdoorspace', 'sparse_outdoorspace', 'sparse_loft', 'sparse_diningroom', 'sparse_balcony', 'sparse_highspeedinternet', 'sparse_parkingspace', 'sparse_terrace', 'sparse_swimmingpool', 'west', 'sparse_roofdeck', 'sparse_actualapt', 'sparse_wheelchairaccess', 'sparse_newconstruction', 'Zero_building_id', 'sparse_simplex', 'sparse_patio', 'sparse_garden', 'sparse_multilevel', 'sparse_hardwood', 'sparse_shorttermallowed', 'south', 'sparse_stainlesssteelappliances', 'sparse_fireplace', 'sparse_highceiling', 'sparse_renovated', 'sparse_liveinsuper', 'sparse_storage', 'sparse_garage', 'sparse_dryerinunit', 'sparse_outdoorareas', 'sparse_petsok', 'sparse_lndrybldg', 'sparse_concierge', 'sparse_new', 'sparse_highceilings', 'sparse_onsitelaundry', 'sparse_centrala', 'sparse_flex3', 'sparse_photos', 'sparse_view', 'sparse_publicoutdoor', 'sparse_allutilitiesincluded', 'sparse_residentslounge', 'sparse_newlyrenovated', 'sparse_washerinunit', 'sparse_onsitegarage', 'sparse_assignedparkingspace', 'north', 'sparse_washer', 'sparse_light', 'sparse_dryer', 'sparse_lowrise', 'sparse_sublet', 'sparse_granitekitchen', 'sparse_elev', 'sparse_virtualdoorman', 'sparse_sundeck', 'sparse_rooftopdeck', 'sparse_wallsofwindows', 'sparse_sharesok', 'sparse_duplex', 'sparse_nopets', 'sparse_cable', 'sparse_microwave', 'sparse_wifiaccess', 'sparse_walkincloset', 'sparse_petsonapproval', 'sparse_pool', 'sparse_eatinkitchen', 'sparse_marblebath', 'sparse_live', 'sparse_sauna', 'sparse_greenbuilding', 'sparse_exposedbrick', 'sparse_largelivingroom', 'sparse_bikeroom', 'sparse_highrise', 'sparse_laundry', 'sparse_privateroofdeck', 'sparse_laundryroom', 'sparse_commonbackyard', 'sparse_privatebackyard', 'sparse_parking', 'sparse_privateparking', 'sparse_childrensplayroom', 'sparse_privatebalcony', 'sparse_indoorpool']
leaky = ['manager_worths', 'building_worths', 'price_by_area', 'real_minus_expected_price', 'real_over_expected_price', 'price_by_address1', 'worth_by_address1', 'price_by_BoroCode', 'worth_by_BoroCode', 'price_by_NTACode', 'worth_by_NTACode']
excluded = [] #'BoroCode', 'NTACode'] #, 'price_per_bedroom', 'price_per_bathroom', 'price_per_bed_and_bath']
def filter_data_to_columns(data):
    return data #[[col for col in current_filter if col not in leaky and col not in excluded]]

# Run CV

In [91]:
class FakeDMatrix:
    def __init__(self, data, labels=None):
        self.data = data
        self.labels = labels
        self.num = len(data)

    def num_row(self):
        return self.num

    def slice(self, rindex):
        indices = np.zeros(self.num, dtype=np.bool)
        for index in rindex:
            indices[index] = True
        return FakeDMatrix(data=self.data[indices], labels=self.labels[indices])

In [4]:
%%time    
SEED = 777
NFOLDS = 3
y_map = {'low': 2, 'medium': 1, 'high': 0}

params = {
    'eta':.01,
    'colsample_bytree':.8,
    'subsample':.8,
    'seed':0,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

processed_train = filter_data_to_columns(pd.read_json('data/train.json'))
y_train = pd.read_json('data/train_interest.json')

dtrain = FakeDMatrix(data=processed_train, labels=y_train)
processed_train = None
y_train = None

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200, fpreproc=fpreproc_leaky)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])
print (best_rounds)

[0]	train-rmse:2028.02+4.95931	test-rmse:2026.61+63.9387
1577.82290675
[0]	train-rmse:2042.58+48.7314	test-rmse:2040.43+112.343
1572.832428
[0]	train-rmse:2040.83+24.4325	test-rmse:2040.08+66.6658
[200]	train-rmse:1576.46+25.7335	test-rmse:1575.47+74.8311
1575.4721375
[0]	train-mlogloss:1.09149+4.76189e-05	test-mlogloss:1.09179+3.25201e-05
[200]	train-mlogloss:0.590529+0.00206352	test-mlogloss:0.632937+0.00437172
[400]	train-mlogloss:0.509094+0.00245771	test-mlogloss:0.577507+0.00577111
[600]	train-mlogloss:0.473041+0.00278344	test-mlogloss:0.5614+0.00599762
[800]	train-mlogloss:0.448223+0.00272998	test-mlogloss:0.553912+0.00611605
[1000]	train-mlogloss:0.428439+0.00247294	test-mlogloss:0.549505+0.00607799
[1200]	train-mlogloss:0.411654+0.00215984	test-mlogloss:0.546701+0.00602279
[1400]	train-mlogloss:0.396108+0.00176339	test-mlogloss:0.544981+0.00610467
[1600]	train-mlogloss:0.382032+0.00165274	test-mlogloss:0.543897+0.00606365
[1800]	train-mlogloss:0.368611+0.001446	test-mlogloss:0.

- 0.548561666667 for all features (lb 0.54993)
- 0.542629666667 for all features after fixing from all worths being 0

# Run To Submit

In [5]:
%%time
processed_test = filter_data_to_columns(pd.read_json('data/test.json'))
listing_id = pd.read_json('data/test_ids.json').values

dtest = FakeDMatrix(data=processed_test)
processed_test = None

dtrain_final, dtest_final, _ = fpreproc_leaky(dtrain, dtest, None)
dtrain = None
dtest = None

bst = xgb.train(params, dtrain_final, best_rounds)
dtrain_final = None

preds = bst.predict(dtest_final)
# save for column names
#dtest_final = None

preds = pd.DataFrame(preds)
preds.columns = ['high', 'medium', 'low']
preds['listing_id'] = listing_id
preds.to_csv('data/my_preds.csv', index=None)

importance = bst.get_fscore()
feature_importance = [(feature, (importance[feature])) for feature in importance]
print (sorted(feature_importance, key=lambda x: -x[1]))

[0]	train-rmse:2001.04+15.3439	test-rmse:1999.82+26.0661
1772.45883175
[('bc_price', 16597), ('manager_worths', 14684), ('price_by_NTACode', 13343), ('latitude', 12299), ('longitude', 12051), ('desc_words_length', 11477), ('manager_id', 11136), ('price_by_address1', 11125), ('Yday', 10884), ('address1', 10458), ('building_worths', 10298), ('building_id', 10142), ('desc_letters_count', 9832), ('real_minus_expected_price', 9815), ('price_by_area', 9742), ('real_over_expected_price', 9374), ('Day', 9066), ('worth_by_address1', 8782), ('hour', 7722), ('price_per_bathroom', 7590), ('desc_words_count', 7481), ('price_per_bed_and_bath', 7387), ('worth_by_NTACode', 6993), ('photos_count', 6730), ('NTACode', 6324), ('price_by_BoroCode', 6098), ('features_count', 5668), ('Wday', 4445), ('price_per_bedroom', 3903), ('sparse_nofee', 3056), ('bedrooms', 2833), ('sparse_furnished', 1865), ('bathrooms', 1702), ('sparse_laundryinbuilding', 1425), ('sparse_hardwoodfloors', 1132), ('sparse_laundryinunit

In [8]:
#best_preds = pd.read_csv('data/my_best_preds.csv')
#truths = np.argmax(best_preds[['high', 'medium', 'low']].values, axis=1)
#should be < 0.5 for a good submission
#log_loss(truths, preds[['high', 'medium', 'low']].values)

1.0488940684429542

# Add leaky for CV

In [92]:
processed_train = filter_data_to_columns(pd.read_json('data/train.json'))

In [93]:
#%%time
for column in leaky:
    processed_train[column] = 0
folds = 5
full_length = len(processed_train)
entries = int(full_length / folds)
full_index = processed_train.index
fold_indices = []
fold_indices_keep = []
fold_indices_throw = []
for i in range(folds):
    indices = random.sample(list(full_index), entries)
    fold_indices.append(indices)
    full_index = full_index.drop(indices)

    full_index_keep = np.zeros((full_length), dtype=np.bool)
    full_index_throw = np.ones((full_length), dtype=np.bool)
    for j in fold_indices[i]:
        full_index_keep[j] = True
        full_index_throw[j] = False
    fold_indices_throw.append(full_index_throw)
    fold_indices_keep.append(full_index_keep)
    
    dtrain = FakeDMatrix(data=processed_train[full_index_throw], labels=y_train[full_index_throw])
    dtest = FakeDMatrix(data=processed_train[full_index_keep])
    _, fold_with_leaky, _ = fpreproc_leaky_dataframe(dtrain, dtest)
    processed_train.loc[sorted(indices), leaky] = fold_with_leaky[leaky].values

[0]	train-rmse:1962.29+8.91344	test-rmse:1962.28+26.9893
1566.46768175
[0]	train-rmse:1966.43+18.7383	test-rmse:1966.19+43.832
1566.7273255
[0]	train-rmse:1997.05+63.7993	test-rmse:1999.45+79.7157
1563.3525695
[0]	train-rmse:2004.54+23.8324	test-rmse:2003.77+56.2566
1565.869293
[0]	train-rmse:1967.61+23.4372	test-rmse:1967.12+69.144
[200]	train-rmse:1565.8+20.4719	test-rmse:1565.22+61.8834
1565.22198475


In [94]:
processed_train

,BoroCode,Day,Month,NTACode,Wday,Yday,Zero_building_id,Zero_manager_id,avenue,bathrooms,...,building_worths,price_by_area,real_minus_expected_price,real_over_expected_price,price_by_address1,worth_by_address1,price_by_BoroCode,worth_by_BoroCode,price_by_NTACode,worth_by_NTACode
0,2,24,6,45,4,176,0,0,1,1.5,...,0.000000,1.000000,-1592.625488,0.653221,0,0,0.547199,0.486303,0.561162,0.441176
1,0,12,6,90,6,164,0,0,1,1.0,...,0.100000,1.000000,1696.736572,1.450270,0,0,1.120900,0.357769,1.076860,0.286633
2,0,14,4,105,3,105,1,0,1,1.0,...,0.110893,1.000000,-1059.055664,0.648044,0,0,0.799910,0.357769,0.889355,0.341168
3,2,13,4,11,2,104,0,0,1,1.0,...,0.000000,1.000000,-650.430420,0.754594,0,0,0.547199,0.486303,0.746299,0.892157
4,0,12,4,93,1,103,0,0,0,1.0,...,0.000000,1.000000,171.378174,1.055219,0,0,1.343439,0.354811,1.381759,0.362030
5,0,19,4,92,1,110,0,0,0,1.0,...,0.200000,1.000000,146.135986,1.046336,0,0,1.353694,0.354807,1.028422,0.257399
6,0,6,4,91,2,97,0,0,0,1.0,...,0.316327,1.472196,737.216797,1.226642,0,0,1.636739,0.354807,1.429569,0.349218
7,0,26,4,98,1,117,0,0,0,1.0,...,0.500000,1.000000,-429.581543,0.858158,0,0,1.066137,0.354811,1.150486,0.445681
8,3,15,4,125,4,106,0,0,0,1.0,...,0.737500,1.000000,-182.442627,0.916404,0,0,0.566940,0.573653,0.952038,0.888889
9,2,4,6,34,5,156,0,0,0,1.0,...,0.000000,1.000000,1676.426758,1.691717,0,0,2.243516,0.492616,1.598708,0.269565


In [95]:
processed_train = processed_train.drop('price', axis=1)
processed_train.to_json('data/train_with_leaky_cv.json', orient='records')

# Add leaky feats and save

In [6]:
processed_train = filter_data_to_columns(pd.read_json('data/train.json'))
y_train = pd.read_json('data/train_interest.json')
dtrain = FakeDMatrix(data=processed_train, labels=y_train)

processed_test = filter_data_to_columns(pd.read_json('data/test.json'))
listing_id = pd.read_json('data/test_ids.json').values
dtest = FakeDMatrix(data=processed_test)
train, test, y_train = fpreproc_leaky_dataframe(dtrain, dtest)
train.to_json('data/train_leaky.json', orient='records')
test.index = range(len(test))
test.to_json('data/test_leaky.json', orient='records')

[0]	train-rmse:2001.94+15.2958	test-rmse:2002.47+35.3504
1772.8644105
